In [1]:
import zipfile
import requests
import os
import json
import polars as pl
from dotenv import load_dotenv
from collections import Counter

load_dotenv()

zip_file_path = "./export/Account Settings Dec 18 2024.zip"

extract_to_path = "./extracted_files"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

In [2]:
# read csv from extracted files diary with polars

df = pl.read_csv(f"{extract_to_path}/diary.csv").filter(pl.col("Watched Date") > pl.lit("2024-01-01"))

In [3]:
df = df.with_columns( 
    pl.col("Date").str.to_datetime("%Y-%m-%d"),
    pl.col("Watched Date").str.to_datetime("%Y-%m-%d"),
    pl.concat_str(
        [
            pl.col("Name"),
            pl.col("Year"),
        ],
        separator="_",
    ).alias("title_year")
)

df

Date,Name,Year,Letterboxd URI,Rating,Rewatch,Tags,Watched Date,title_year
datetime[μs],str,i64,str,f64,str,str,datetime[μs],str
2024-01-05 00:00:00,"""Fallen Leaves""",2023,"""https://boxd.it/5uQmgp""",3.5,null,null,2024-01-04 00:00:00,"""Fallen Leaves_2023"""
2024-01-06 00:00:00,"""Heat""",1995,"""https://boxd.it/5vpfCt""",5.0,null,null,2024-01-05 00:00:00,"""Heat_1995"""
2024-01-08 00:00:00,"""The Boy and the Heron""",2023,"""https://boxd.it/5wOdu7""",4.5,null,null,2024-01-07 00:00:00,"""The Boy and the Heron_2023"""
2024-01-08 00:00:00,"""Anatomy of a Fall""",2023,"""https://boxd.it/5x4onX""",4.5,null,null,2024-01-08 00:00:00,"""Anatomy of a Fall_2023"""
2024-01-10 00:00:00,"""The Holdovers""",2023,"""https://boxd.it/5ybXH7""",4.5,null,null,2024-01-09 00:00:00,"""The Holdovers_2023"""
…,…,…,…,…,…,…,…,…
2024-11-18 00:00:00,"""Submarine""",2010,"""https://boxd.it/7PT77N""",5.0,null,null,2024-11-17 00:00:00,"""Submarine_2010"""
2024-11-19 00:00:00,"""Ferris Bueller's Day Off""",1986,"""https://boxd.it/7QqY5B""",5.0,null,null,2024-11-18 00:00:00,"""Ferris Bueller's Day Off_1986"""
2024-11-22 00:00:00,"""Wicked""",2024,"""https://boxd.it/7ROjtJ""",4.5,null,null,2024-11-21 00:00:00,"""Wicked_2024"""


In [4]:
CACHE_FILE = "movie_cache.json"

In [5]:
def load_cache():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_cache(cache):
    with open(CACHE_FILE, 'w') as f:
        json.dump(cache, f, indent=4)

In [6]:
def get_movie_data(title, year, api_key):
    parsed_title = title.replace(" ", "+")

    cache = load_cache()
    key = f"{title}_{year}"

    if key in cache:
        print(f"Cache hit per '{title}' ({year})")
        return cache[key]

    url = f"http://www.omdbapi.com/?&t={parsed_title}&y={year}&apikey={api_key}"

    response = requests.get(url)
    data = response.json()

    if response.status_code == 200 and data.get("Response") == "True":
        cache[key] = data
        save_cache(cache)
        return data
    else:
        print(f"Errore: {data.get('Error')}")
        return None

In [7]:
API_KEY = os.getenv("API_KEY")

get_movie_data("Submarine", 2010, API_KEY)

Cache hit per 'Submarine' (2010)


{'Title': 'Submarine',
 'Year': '2010',
 'Rated': 'R',
 'Released': '18 Mar 2011',
 'Runtime': '97 min',
 'Genre': 'Comedy, Drama, Romance',
 'Director': 'Richard Ayoade',
 'Writer': 'Richard Ayoade, Joe Dunthorne',
 'Actors': 'Craig Roberts, Sally Hawkins, Paddy Considine',
 'Plot': '15-year-old Oliver Tate has two objectives: to lose his virginity before his next birthday, and to extinguish the flame between his mother and an ex-lover who has resurfaced in her life.',
 'Language': 'English',
 'Country': 'United Kingdom, United States',
 'Awards': 'Nominated for 1 BAFTA Award6 wins & 17 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTQ5ODMxNTIyNV5BMl5BanBnXkFtZTcwNjQ1ODgyNQ@@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.3/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '88%'},
  {'Source': 'Metacritic', 'Value': '76/100'}],
 'Metascore': '76',
 'imdbRating': '7.3',
 'imdbVotes': '97,537',
 'imdbID': 'tt1440292',
 'Type': 'movi

In [8]:
def get_metatadat(diary_df, api_key):
    for t, y in diary_df[['Name', 'Year']].rows():
        print(f"Getting data for {t} ({y})")
        get_movie_data(t, y, api_key)


In [9]:
def extract_metadata(cache, title_year, param):
        temp_metadata = []
        for ty in title_year:
            temp_data = cache.get(ty)
            if temp_data is not None:
                temp_metadata.append(cache.get(ty)[param])
            else:
                temp_metadata.append(None)
        return pl.Series("metadata", temp_metadata)

cache = load_cache()

df = df.with_columns(
    (pl.struct(["title_year"]).map_batches(lambda movie: extract_metadata(cache, movie.struct.field("title_year"), "Genre"))).str.split(", ").alias("Genre"),
    (pl.struct(["title_year"]).map_batches(lambda movie: extract_metadata(cache, movie.struct.field("title_year"), "Runtime"))).alias("Runtime"),
    (pl.struct(["title_year"]).map_batches(lambda movie: extract_metadata(cache, movie.struct.field("title_year"), "Director"))).alias("Director"),
    (pl.struct(["title_year"]).map_batches(lambda movie: extract_metadata(cache, movie.struct.field("title_year"), "Writer"))).str.split(", ").alias("Writer"),
    (pl.struct(["title_year"]).map_batches(lambda movie: extract_metadata(cache, movie.struct.field("title_year"), "Actors"))).str.split(", ").alias("Actors"),
    (pl.struct(["title_year"]).map_batches(lambda movie: extract_metadata(cache, movie.struct.field("title_year"), "Country"))).str.split(", ").alias("Country"),
    (pl.struct(["title_year"]).map_batches(lambda movie: extract_metadata(cache, movie.struct.field("title_year"), "Language"))).str.split(", ").alias("Language"),
    (pl.struct(["title_year"]).map_batches(lambda movie: extract_metadata(cache, movie.struct.field("title_year"), "Metascore"))).str.to_integer(strict=False).alias("Metascore"),
    (pl.struct(["title_year"]).map_batches(lambda movie: extract_metadata(cache, movie.struct.field("title_year"), "imdbRating"))).str.to_decimal().alias("imdbRating"),
)

df = df.with_columns(
    ((pl.col("Runtime").str.replace(" min", "").str.to_integer())).alias("Runtime_normalized"),
    ((pl.col("Metascore")) * 5/ 100).alias("Normalized_Metascore"),
    ((pl.col("imdbRating")) * 5/ 10).alias("Normalized_IMDB"),
    (pl.col("Rating") - (pl.col("Metascore")) * 5/ 100).abs().alias("Rating_Difference_Metascore"),
    (pl.col("Rating") - (pl.col("imdbRating")) * 5/ 10).abs().alias("Rating_Difference_IMDB"),
)
        

In [27]:
df.head(2)

Date,Name,Year,Letterboxd URI,Rating,Rewatch,Tags,Watched Date,title_year,Genre,Runtime,Director,Writer,Actors,Country,Language,Metascore,imdbRating,Runtime_normalized,Normalized_Metascore,Normalized_IMDB,Rating_Difference_Metascore,Rating_Difference_IMDB
datetime[μs],str,i64,str,f64,str,str,datetime[μs],str,list[str],str,str,list[str],list[str],list[str],list[str],i64,"decimal[*,1]",i64,f64,"decimal[*,6]",f64,f64
2024-01-05 00:00:00,"""Fallen Leaves""",2023,"""https://boxd.it/5uQmgp""",3.5,null,null,2024-01-04 00:00:00,"""Fallen Leaves_2023""","[""Comedy"", ""Drama"", ""Romance""]","""81 min""","""Aki Kaurismäki""","[""Aki Kaurismäki""]","[""Alma Pöysti"", ""Jussi Vatanen"", ""Janne Hyytiäinen""]","[""Finland"", ""Germany""]","[""Finnish"", ""Arabic""]",86,7.3,81,4.3,3.650000,0.8,0.15
2024-01-06 00:00:00,"""Heat""",1995,"""https://boxd.it/5vpfCt""",5.0,null,null,2024-01-05 00:00:00,"""Heat_1995""","[""Action"", ""Crime"", ""Drama""]","""170 min""","""Michael Mann""","[""Michael Mann""]","[""Al Pacino"", ""Robert De Niro"", ""Val Kilmer""]","[""United States""]","[""English"", ""Spanish""]",76,8.3,170,3.8,4.150000,1.2,0.85


In [28]:
def get_top_from_list(df, column, top=5):
    items = []
    for item_list in df.select(column).rows():
        if item_list[0]:
            for i in item_list[0]:
                items.append(i)

    item_counts = Counter(items)

    for item, count in item_counts.most_common(top):
        print(f"{item}: {count}")

# Results

In [30]:
# Longest movies
df.select("Name", "Runtime_normalized").drop_nulls().sort("Runtime_normalized", descending=True).head(5)

Name,Runtime_normalized
str,i64
"""A Brighter Summer Day""",237
"""Lawrence of Arabia""",227
"""The Sound of Music""",172
"""Heat""",170
"""Dune: Part Two""",166


In [31]:
# Total screen time
df.select("Runtime_normalized").sum()/60

Runtime_normalized
f64
177.766667


In [35]:
# Top 5 month screen time
df.group_by_dynamic("Watched Date", every="1mo").agg(pl.col("Runtime_normalized").sum()/60).sort("Runtime_normalized", descending=True).head(5)

Watched Date,Runtime_normalized
datetime[μs],f64
2024-07-01 00:00:00,23.316667
2024-10-01 00:00:00,20.666667
2024-09-01 00:00:00,17.85
2024-08-01 00:00:00,17.7
2024-03-01 00:00:00,17.416667


In [38]:
# Total number of movies
df.select("Name").count()

Name
u32
90


In [51]:
# Total number of directors
df.select("Director").n_unique()

79

In [46]:
# Top directors
df.group_by("Director").agg(pl.col("Name").count().alias("count")).sort("count", descending=True).head(5)

Director,count
str,u32
"""Billy Wilder""",3
"""Yorgos Lanthimos""",2
"""Akira Kurosawa""",2
"""Sidney Lumet""",2
"""Will Gluck""",2


In [40]:
# Top actors
get_top_from_list(df, "Actors", 10)

Al Pacino: 3
Robert De Niro: 2
Sandra Hüller: 2
Emma Stone: 2
Willem Dafoe: 2
Zendaya: 2
Sam Neill: 2
Roy Scheider: 2
Mia Goth: 2
Tony Leung Chiu-wai: 2


In [54]:
# Top writers
get_top_from_list(df, "Writer", 10)

Billy Wilder: 3
Ti West: 2
Ingmar Bergman: 2
I.A.L. Diamond: 2
Hideo Oguni: 2
Richard Linklater: 2
Aki Kaurismäki: 1
Michael Mann: 1
Hayao Miyazaki: 1
Justine Triet: 1


In [61]:
# Top genres
get_top_from_list(df, "Genre", 10)

Drama: 60
Comedy: 33
Romance: 19
Action: 14
Crime: 14
Adventure: 14
Horror: 12
Thriller: 11
Fantasy: 10
Animation: 7


In [52]:
# Movies by country
get_top_from_list(df, "Country", 10)

United States: 63
United Kingdom: 14
Japan: 12
France: 12
Canada: 9
Italy: 9
New Zealand: 5
Germany: 3
Australia: 3
Ireland: 2


In [53]:
# Movies by language
get_top_from_list(df, "Language", 10)

English: 70
French: 17
Japanese: 11
Italian: 11
Spanish: 8
German: 8
Mandarin: 4
Cantonese: 4
Latin: 4
Korean: 3


In [60]:
# Top movies by my rating
for m, r in df.select("Name", "Rating").drop_nulls().sort("Rating", descending=True).rows():
    if r == 5:
        print(m, r)

# Perfect Days 5.0
# Past Lives 5.0
# Dune: Part Two 5.0
# The Thing 5.0
# Ferris Bueller's Day Off 5.0


Heat 5.0
Perfect Days 5.0
Past Lives 5.0
Dune: Part Two 5.0
Possession 5.0
The Thing 5.0
Die Hard 5.0
Superbad 5.0
The Seventh Seal 5.0
Come and See 5.0
The Apartment 5.0
Paths of Glory 5.0
Stalker 5.0
All That Jazz 5.0
Taste of the Sea 5.0
Before Midnight 5.0
Some Like It Hot 5.0
The Sound of Music 5.0
Groundhog Day 5.0
Rushmore 5.0
Ran 5.0
The Wizard of Oz 5.0
Look Back 5.0
A Woman Under the Influence 5.0
Submarine 5.0
Ferris Bueller's Day Off 5.0


In [18]:
# Top movies by Metascore
df.select("Name", "Metascore").drop_nulls().sort("Metascore", descending=True).head(5)

Name,Metascore
str,i64
"""Lawrence of Arabia""",100
"""Some Like It Hot""",98
"""Ran""",97
"""Portrait of a Lady on Fire""",95
"""Past Lives""",94


In [19]:
df.select("Name", "Rating_Difference_Metascore", "Rating", "Normalized_Metascore").drop_nulls().sort("Rating_Difference_Metascore", descending=True).head(5)

Name,Rating_Difference_Metascore,Rating,Normalized_Metascore
str,f64,f64,f64
"""Friday the 13th""",2.9,4.0,1.1
"""The Thing""",2.15,5.0,2.85
"""Bullet Train""",2.05,4.5,2.45
"""Ferris Bueller's Day Off""",1.95,5.0,3.05
"""The Sound of Music""",1.85,5.0,3.15


In [20]:
# Most controversial ratings
df.select("Name", "Rating_Difference_IMDB", "Rating", "Normalized_IMDB").drop_nulls().sort("Rating_Difference_IMDB", descending=True).head(5)

Name,Rating_Difference_IMDB,Rating,Normalized_IMDB
str,f64,f64,"decimal[*,6]"
"""Salò, or the 120 Days of Sodom""",2.4,0.5,2.900000
"""Possession""",1.35,5.0,3.650000
"""Submarine""",1.35,5.0,3.650000
"""Superbad""",1.2,5.0,3.800000
"""Rushmore""",1.2,5.0,3.800000


In [24]:
df.group_by("Year").agg(pl.col("Name").count().alias("count")).sort("count", descending=True)

Year,count
i64,u32
2023,16
2024,13
2022,4
1993,3
1975,3
…,…
2021,1
1976,1
1959,1
